# Setup

In [46]:
from IPython.display import display, Markdown
from textwrap import fill

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, pipeline

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.sequential import SimpleSequentialChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.schema import AIMessage, HumanMessage
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonAstREPLTool

In [2]:
# MODEL_ID = "TheBloke/Llama-2-13b-Chat-GPTQ"
MODEL_ID = "meta-llama/Llama-2-7b-chat-hf"

# Model (LLM) Wrappers

In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto"
)

generation_config = GenerationConfig.from_pretrained(MODEL_ID)
generation_config.max_new_tokens = 256
generation_config.temperature = 1e-4
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

# Create the HF pipeline
llama_pipeline = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config
)

llama_llm = HuggingFacePipeline(pipeline=llama_pipeline)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
result = llama_llm.invoke("Give the difference between CNN model and ViT")
display(Markdown(result))

Give the difference between CNN model and ViT model?

CNN (Convolutional Neural Network) and VIT (Vision Transformer) are both deep learning architectures used for image classification tasks. While they share some similarities, there are also significant differences between them:

1. Architecture: The most obvious difference is in their architecture. A CNN consists of multiple convolutional layers followed by pooling layers, normalization layers, and finally, fully connected layers for classification. On the other hand, a VIT model uses a self-attention mechanism to process input images as a sequence of patches instead of using convolutional layers.
2. Patch Embedding: In a VIT model, each image is divided into fixed-size non-overlapping patches, and each patch is embedded into a higher-dimensional space using a learnable embedding layer. This allows the model to capture long-range dependencies between different parts of the image. In contrast, a CNN typically uses convolutional filters to extract features from small regions of an image.
3. Number of Parameters: Due to its complex architecture, a VIT model has significantly more parameters than a CNN. For example, a large VIT model may have over 100 million parameters, while

# Prompt and Prompt Templates

In [16]:
template = """
<s>[INST] <<SYS>>
Act as a Machine Learning engineer who is teaching high school students. 
<<SYS>>
{text}[/INST]
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template
)

In [17]:
text = "Explain ViT in 2-3 sentences"
print(prompt.format(text=text))


<s>[INST] <<SYS>>
Act as a Machine Learning engineer who is teaching high school students. 
<<SYS>>
Explain ViT in 2-3 sentences[/INST]



# Create a Chain

In [18]:
chain = LLMChain(llm=llama_llm, prompt=prompt)
result = chain.invoke(text)
print(result)

{'text': "\n<s>[INST] <<SYS>>\nAct as a Machine Learning engineer who is teaching high school students. \n<<SYS>>\nExplain ViT in 2-3 sentences[/INST]\n\nHey there, young minds! *adjusts glasses* Today we're going to explore an exciting topic in the world of machine learning: Vision Transformers (ViT). Essentially, ViT is a type of neural network that uses a transformer architecture to process visual data, like images, and recognize patterns within them. Unlike traditional convolutional neural networks (CNNs), which rely on convolutional layers to extract features from images, ViT divides the image into fixed-size non-overlapping patches and linearly embeds them into a sequence of vectors. This allows the model to learn long-range dependencies between different parts of the image, resulting in improved performance on tasks such as object detection and segmentation. Pretty cool, right? 🤖"}


# Chaining Chains

In [19]:
example_template = """
<s> [INST] Use the summary {summary} and give 3 examples of practical applications 
with 1 sentence explainingeach [/INST]
"""
example_prompt = PromptTemplate(
    input_variables=["summary"],
    template=example_template
)
example_chain = LLMChain(llm=llama_llm, prompt=example_prompt)

In [22]:
# Combine both chains
multi_chain = SimpleSequentialChain(chains=[chain, example_chain], verbose=False)
result = multi_chain.invoke(text)
print(result)

{'input': 'Explain ViT in 2-3 sentences', 'output': '\n<s> [INST] Use the summary \n<s>[INST] <<SYS>>\nAct as a Machine Learning engineer who is teaching high school students. \n<<SYS>>\nExplain ViT in 2-3 sentences[/INST]\n\nHey there, young minds! *adjusts glasses* Today we\'re going to explore an exciting topic in the world of machine learning: Vision Transformers (ViT). Essentially, ViT is a type of neural network that uses a transformer architecture to process visual data, like images, and recognize patterns within them. Unlike traditional convolutional neural networks (CNNs), which rely on convolutional layers to extract features from images, ViT divides the image into fixed-size non-overlapping patches and linearly embeds them into a sequence of vectors. This allows the model to learn long-range dependencies between different parts of the image, resulting in improved performance on tasks such as object detection and segmentation. Pretty cool, right? 🤖 and give 3 examples of prac

# Chatbot

In [26]:
template = "Act as an experienced high school taecher that teaches {subject}. Always give examples adn analogies."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(template),
        HumanMessage(content="Hello teacher!"),
        AIMessage(content="Welcome everyone!"),
        HumanMessagePromptTemplate.from_template(human_template)
    ]
)

messages = chat_prompt.format_messages(
    subject="Machine Learning",
    text="What is the most powerful AI Model?"
)
messages

[SystemMessage(content='Act as an experienced high school taecher that teaches Machine Learning. Always give examples adn analogies.'),
 HumanMessage(content='Hello teacher!'),
 AIMessage(content='Welcome everyone!'),
 HumanMessage(content='What is the most powerful AI Model?')]

In [27]:
result = llama_llm.predict_messages(messages)
print(result.content)

/Users/mmenendezg/Developer/Projects/llama-project/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


System: Act as an experienced high school taecher that teaches Machine Learning. Always give examples adn analogies.
Human: Hello teacher!
AI: Welcome everyone!
Human: What is the most powerful AI Model?
AI: Ah, a great question! *adjusts glasses* The most powerful AI model depends on various factors such as the task at hand and the data available. However, some of the most popular and widely used models include:

1. Neural Networks: These are modeled after the structure of the human brain and have been instrumental in achieving state-of-the-art performance in various applications such as image classification, speech recognition, and natural language processing.
2. Support Vector Machines (SVMs): SVMs are robust and effective for classification tasks, especially when dealing with large datasets. They work by finding the hyperplane that maximally separates the classes in the feature space.
3. Random Forests: This ensemble learning method combines multiple decision trees to improve the a

# Simple Retrieval Augmented Generation (RAG)

In [31]:
md_file = "../docs/bitcoin.md"

loader = UnstructuredMarkdownLoader(md_file)
docs = loader.load()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mmenendezg/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mmenendezg/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [32]:
len(docs)

1

In [34]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

29

In [37]:
embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": "mps"},
    encode_kwargs={"normalize_embeddings": True}
)

query_result = embeddings.embed_query(texts[0].page_content)
print(len(query_result))

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

1024


In [39]:
db = Chroma.from_documents(texts, embeddings, persist_directory="../data/interim")
results = db.similarity_search("proof-of-work majoirty decision making", k=2)
print(results[0].page_content)

The proof-of-work also solves the problem of determining representation
in majority decision making. If the majority were based on
one-IP-address-one-vote, it could be subverted by anyone able to
allocate many IPs. Proof-of-work is essentially one-CPU-one-vote. The
majority decision is represented by the longest chain, which has the
greatest proof-of-work effort invested in it. If a majority of CPU power
is controlled by honest nodes, the honest chain will grow the fastest
and outpace any competing chains. To modify a past block, an attacker
would have to redo the proof-of-work of the block and all blocks after
it and then catch up with and surpass the work of the honest nodes. We
will show later that the probability of a slower attacker catching up
diminishes exponentially as subsequent blocks are added.


In [41]:
template = """
<s>[INST] <<SYS>>
Act as a cryptocurrency expert. Use the following information to answer the question at the end.
<</SYS>>
 
{context}
 
{question} [/INST]
"""
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

qa_chain = RetrievalQA.from_chain_type(
    llm=llama_llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

result = qa_chain.invoke(
    "How does proof-of-work solves the majority deicison making problem?"
)
print(result["result"].strip())

/Users/mmenendezg/Developer/Projects/llama-project/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


<s>[INST] <<SYS>>
Act as a cryptocurrency expert. Use the following information to answer the question at the end.
<</SYS>>
 
The proof-of-work also solves the problem of determining representation
in majority decision making. If the majority were based on
one-IP-address-one-vote, it could be subverted by anyone able to
allocate many IPs. Proof-of-work is essentially one-CPU-one-vote. The
majority decision is represented by the longest chain, which has the
greatest proof-of-work effort invested in it. If a majority of CPU power
is controlled by honest nodes, the honest chain will grow the fastest
and outpace any competing chains. To modify a past block, an attacker
would have to redo the proof-of-work of the block and all blocks after
it and then catch up with and surpass the work of the honest nodes. We
will show later that the probability of a slower attacker catching up
diminishes exponentially as subsequent blocks are added.

For our timestamp network, we implement the proof-of-wor

In [43]:
result = qa_chain.invoke(
    "Summarize the privacy compared to the traditional banking model in 2-3 sentences.",
    max_new_tokens=250
)
print(fill(result["result"].strip(), width=80))

<s>[INST] <<SYS>> Act as a cryptocurrency expert. Use the following information
to answer the question at the end. <</SYS>>   Privacy  The traditional banking
model achieves a level of privacy by limiting access to information to the
parties involved and the trusted third party. The necessity to announce all
transactions publicly precludes this method, but privacy can still be maintained
by breaking the flow of information in another place: by keeping public keys
anonymous. The public can see that someone is sending an amount to someone else,
but without information linking the transaction to anyone. This is similar to
the level of information released by stock exchanges, where the time and size of
individual trades, the \"tape\", is made public, but without telling who the
parties were.  [^7]: R.C. Merkle, \"Protocols for public key cryptosystems,\" In
Proc.     1980 Symposium on Security and Privacy, IEEE Computer Society, pages
122-133, April 1980.  [^8]: H.  Massias, X.S. Avila, an

# Agents

In [47]:
agent = create_python_agent(
    llm=llama_llm,
    tool=PythonAstREPLTool(),
    verbose=True,
)

result = agent.invoke("Calculate all the prime numbers between 0 and 100")



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Parsing LLM output produced both a final answer and a parse-able action:: You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.


python_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.

Use the followi